<a href="https://colab.research.google.com/github/Hoshi54/projects_ML/blob/main/SHIFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transform
import torchvision
from sklearn.model_selection import train_test_split


train_transform = transform.Compose(
    [
        transform.Resize((244,244)),
        transform.RandomResizedCrop(244),
        transform.RandomRotation(degrees = [10,15]),
        transform.RandomHorizontalFlip(),
        transform.ToTensor()
    ]
)

test_transform = transform.Compose(
    [
        transform.Resize((244,244)),
        transform.CenterCrop(244),
        transform.ToTensor()
    ]
)

df = pd.read_csv('/content/sign_mnist.csv')

train_df, test_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)

train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)

class SignDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data.iloc[idx, 1:].to_numpy().reshape(1, 28, 28)
        label = self.data.iloc[idx]['label']
        return torch.tensor(image).float(), torch.tensor(label)


train_dataset = SignDataset('/content/train.csv',train_transform)
test_dataset = SignDataset('/content/test.csv',test_transform)
eval_dataset = SignDataset('/content/sign_mnist_test.csv')

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,shuffle = True,
                                           batch_size = 32,num_workers = 2)

test_loader = torch.utils.data.DataLoader(dataset = train_dataset,shuffle = False,
                                           batch_size = 32,num_workers = 2)

eval_loader = torch.utils.data.DataLoader(dataset = eval_dataset,shuffle = False,
                                      batch_size = 32,num_workers = 2)

In [ ]:
class SimpleNet(nn.Module):
  def __init__(self):
    super(SimpleNet,self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 1,out_channels = 64,kernel_size = 5)
    self.batch1 = nn.BatchNorm2d(64)
    self.pool1 = nn.MaxPool2d(kernel_size = 2,stride = 2)
    self.conv2 = nn.Conv2d(in_channels = 64,out_channels = 128,kernel_size = 5)
    self.batch2 = nn.BatchNorm2d(128)
    self.pool2 = nn.MaxPool2d(kernel_size = 2,stride = 2)
    self.conv3 = nn.Conv2d(in_channels = 128,out_channels = 256,kernel_size = 5)
    self.batch3 = nn.BatchNorm2d(256)
    self.pool3 = nn.MaxPool2d(kernel_size = 2,stride = 2)
    self.drop1 = nn.Dropout(0.25)
    self.drop2 = nn.Dropout(0.5)
    self.fc1 = nn.Linear(4 * 4 * 128,2000)
    self.batch3 = nn.BatchNorm1d(2000)
    self.fc2 = nn.Linear(2000,1000)
    self.batch4 = nn.BatchNorm1d(1000)
    self.fc3 = nn.Linear(1000,25)

  def forward(self,x):
    x = self.batch1(self.pool1(F.relu(self.conv1(x))))
    x = self.batch2(self.pool2(F.relu(self.conv2(x))))
    x = self.drop1(x)
    x = x.view(-1,4 * 4 * 128)
    x = self.batch3(F.relu(self.fc1(x)))
    x = self.drop2(x)
    x = self.batch4(F.relu(self.fc2(x)))
    x = self.fc3(x)
    return x


In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary
summary(SimpleNet(),(32,1,28,28))

In [ ]:
import torch.optim as optim
from tqdm import tqdm_notebook
from sklearn.metrics import f1_score
from torch.optim import lr_scheduler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Зaморозка слоёв fc
#fc = [model.fc1,model.fc2,model.fc3]
#for fc in fc:
#  for param in fc.parameters():
#    param.requires_grad = False

model = SimpleNet().to(device)
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

lr_schedul = lr_scheduler.StepLR(optimizer,step_size = 1,gamma = 0.1)
f1_train_l = []
f1_test_l = []
acc_train = []
acc_test = []
loss_train = []
loss_te = []

best_acc = 0
best_model_wts = model.state_dict()
for epoch in range(10):
  train_acc = 0
  total_train = 0
  test_acc = 0
  total_test = 0
  for i,(images,labels) in enumerate(tqdm_notebook(train_loader)):
    images,labels = images.to(device),labels.to(device)

    optimizer.zero_grad()

    output = model(images)
    loss = loss_fn(output,labels)
    _,pred = torch.max(output,1)

    f1_train = f1_score(labels.cpu(), pred.cpu(), average = 'weighted')

    train_acc += (pred == labels).sum().item()
    total_train += labels.size(0)
    loss.backward()
    optimizer.step()

  with torch.no_grad():
    for i,(images,labels) in enumerate(tqdm_notebook(test_loader)):
        images = images.to(device)
        labels = labels.to(device)

        output = model(images)
        _,pred = torch.max(output,1)
        test_acc += (pred == labels).sum().item()
        total_test += labels.size(0)
        loss_test = loss_fn(output,labels)
        f1_test = f1_score(labels.cpu(), pred.cpu(), average='weighted')

    if test_acc / total_test > best_acc:
        best_acc = test_acc / total_test
        best_model_wts = model.state_dict()

  acc_train.append(train_acc / total_train)
  acc_test.append(test_acc/total_test)
  loss_train.append(float(loss))
  loss_te.append(float(loss_test))
  f1_train_l.append(f1_train)
  f1_test_l.append(f1_test)
  print('epoch {}/{} loss:{} accuracy_train:{} f1_train:{} | accuracy_test:{} f1_test:{}'.format(epoch + 1,10,loss,train_acc / total_train,f1_train,test_acc/total_test,f1_test))

In [ ]:
for i,(images,labels) in enumerate(tqdm_notebook(eval_loader)):
   labels = labels.type(torch.LongTensor)
   images = images.to(device)
   labels = labels.to(device)

   output = model(images)
   _,pred = torch.max(output,1)
   test_acc += (pred == labels).sum().item()
   total_test += labels.size(0)
   oss_test = loss_fn(output,labels)
   f1_test = f1_score(labels.cpu(), pred.cpu(), average='weighted')

print('accuracy_score:{},f1_test:{}'.format(test_acc / total_test,f1_test))

In [ ]:
import matplotlib.pyplot as plt
plt.title('Потери')
plt.plot(loss_train,label = 'train loss')
plt.plot(loss_te,label = 'test loss')
plt.legend()
plt.show()

In [ ]:
plt.title('Accuracy')
plt.plot(acc_train,label = 'train acc')
plt.plot(acc_test,label = 'test acc')
plt.legend()
plt.show()

In [ ]:
plt.title('f1 score')
plt.plot(f1_train_l,label = 'train f1')
plt.plot(f1_test_l,label = 'test f1')
plt.legend()
plt.show()